### Wstęp do Uczenia Maszynowego 
##### Laboratorium 06

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, precision_score, roc_auc_score, roc_curve
from sklearn.svm import SVC

#### Regresja logistyczna z regularyzacją Elastic Net

Regresja logistyczna estymuje parametry $\boldsymbol{\beta}$ poprzez minimalizację funkcji kosztu (ujemnej log-likelihood) z dodaną karą regularyzacyjną:

$$
\mathcal{L}(\boldsymbol{\beta}) 
= -\frac{1}{n} \sum_{i=1}^{n} 
\Big[ 
y_i \log(p_i) + (1 - y_i) \log(1 - p_i) 
\Big]
+ \lambda \left(
\alpha \|\boldsymbol{\beta}\|_1 + 
\frac{1 - \alpha}{2} \|\boldsymbol{\beta}\|_2^2
\right)
$$

gdzie:

- $p_i = \frac{1}{1 + e^{-(\beta_0 + \mathbf{x}_i^\top \boldsymbol{\beta})}}$  — to prawdopodobieństwo przynależności do klasy pozytywnej,  
- $\lambda > 0$ — współczynnik regularyzacji (im większy, tym silniejsza kara),  
- $\alpha \in [0, 1]$ — współczynnik mieszający między karą $L_1$ i $L_2$:
  - dla $\alpha = 1$ → **Lasso (L1)**,
  - dla $\alpha = 0$ → **Ridge (L2)**,
  - dla $0 < \alpha < 1$ → **Elastic Net** (mieszanka obu).

---
W implementacji `scikit-learn` parametr `C` jest odwrotnością $\lambda$:

$$
C = \frac{1}{\lambda},
$$
a parametrowi $\alpha$ odpowiada `l1_ratio`.



-----
##### *Zadanie 0*
-----
   Dopasuj modele regresji logistycznej z regularyzacją L1 (penalty='l1') dla wartości  
     $$
     C \in \{0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 50, 100\}.
     $$
   - Dla każdej wartości `C` oblicz:
     - wartość wskaźnika AUC na zbiorze testowym,  
     - liczbę niezerowych współczynników modelu.
   - Na podstawie wyników wybierz wartość `C`, która stanowi najlepszy kompromis między jakością predykcji a prostotą modelu.


   Następnie używając wybranego najlepszego `C`, dopasuj modele regresji logistycznej z regularyzacją Elastic Net, zmieniając parametr  
     $$
     \alpha \in \{0.0, 0.1, 0.25, 0.5, 0.75, 0.9,  1.0\}.
     $$
   - Dla każdej wartości $\alpha$ oblicz:
     - wartość AUC na zbiorze testowym,  
     - liczbę niezerowych współczynników.

In [ ]:
credit = fetch_openml(name='credit-g', version=1, as_frame=True)
df = credit.frame

# Zmienna celu: 1 = good, 0 = bad
df['class'] = (df['class'] == 'good').astype(int)

# One-hot encoding zmiennych kategorycznych
X = pd.get_dummies(df.drop(columns='class'), drop_first=True)
y = df['class']

# Podział na zbiory treningowy/testowy
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Standaryzacja danych
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)


# Optymalizacja parametru C dla regularyzacji L1

C_values = np.logspace(-3, 2, 10)  # od 0.001 do 100
results_L1 = []


#### Uwaga

Wartość optymalna parametru `C` uzyskana dla regularyzacji **L1** może stanowić **dobry punkt wyjścia** do dalszej optymalizacji w modelu **Elastic Net**.  
Jednak ze względu na obecność składnika L2 w funkcji kosztu, **optymalne wartości `C` mogą się różnić** między L1 i Elastic Net.  
W praktyce warto:
- przeprowadzić osobną optymalizację obu hiperparametrów (`C`, `α`),  
- lub rozważyć **otoczenie wartości `C_max`** wyznaczonego dla L1, aby sprawdzić stabilność rozwiązania.

---

#### SVM - Support Vector Machine

In [ ]:
# losowanie dwóch zmiennych z rozkładu standardowego normalnego
rng = np.random.default_rng(1)
X = rng.standard_normal((50,2))
y = np.array([-1]*25 + [1]*25)
# przesunięcie punktów o wektor [2, 2]
X[y == 1] += 2


In [ ]:
# wykres punktów
plt.scatter(X[:,0], X[:,1], c = y)

-----
##### *Zadanie 1*
-----
Zbuduj model wektorów podpierających wykorzystując zdefinowane X i y jak zbiór treningowy. Ustaw parametr `C` = 10, `kernel` = 'linear'.

### Wyznaczanie hiperpłaszczyzny (dla dwóch wymiarów)

$<w, x> + b = 0$

$w_1x_1 + w_2x_2 + b = 0$

$w_2x_2 = -w_1x_1 - b$

$x_2 = -\frac{w_1}{w_2}x_1 - \frac{b}{w_2}$


In [ ]:
W = ..
b = ..

A = ..
B = ..

-----
##### *Zadanie 2*
-----
Wyznacz, które punkty są wektorami podpierającymi i zaznacz je na powyższym wykresie.

-----
##### *Zadanie 3*
-----
Zbuduj model wektorów podpierających wykorzystując zdefinowane X i y jak zbiór treningowy. Ustaw parametr `C` = 0.01, `kernel` = 'linear'. Narysuj płaszczyznę rozdzielającą klasy i wskaż wektory podpierające. 

### Wyznaczanie marginesu (dla dwóch wymiarów)

$margin_{magnitude} = \frac{1}{||w||}$

$||w|| = \sqrt{w_1^2 + w_2^2}$

$\hat{w} = \frac{w}{||w||}$


In [ ]:
w_hat = ..

In [ ]:
margin =..

In [ ]:
decision_boundary_points = np.array(list(zip(np.array([-3,3]), A * np.array([-3,3]) + B)))
points_of_line_above = decision_boundary_points + w_hat * margin
points_of_line_below = decision_boundary_points - w_hat * margin

In [ ]:
fig, ax = plt.subplots()
plt.scatter(X[:,0], X[:,1], c = y)
ax.plot(np.array([-3,3]), A * np.array([-3,3]) + B)
# Blue margin line above
plt.plot(points_of_line_above[:, 0], 
         points_of_line_above[:, 1], 
         'b--', 
         linewidth=2)
# Green margin line below
plt.plot(points_of_line_below[:, 0], 
         points_of_line_below[:, 1], 
         'g--',
         linewidth=2)

### Optymalizacja parametru C

In [ ]:
import sklearn.model_selection as skm
kfold = skm.KFold(5,
                  random_state=0,
                  shuffle=True)

In [ ]:
grid = skm.GridSearchCV(svm_linear,
                        {'C':[0.001, 0.01, 0.1, 1, 5, 10, 100]}, 
                         refit=True,
                         cv=kfold,
                         scoring='accuracy')

### Podsumowanie, czyli co warto wiedzieć...


1. Idea regresji liniowej
2. Regresja logistyczna

    a) przekształcenie funkcją sigmoid

    b) wyliczanie współczynników metodą ML

    c) regularyzacja L1 + własności (ograniczenie liczby zmiennych)

    d) regularyzacja L2

3. SVM (liniowe)

    a) wyznaczanie hiperpłaszczyzny, marginesu, wektorów podpierających

    b) przypadek danych nieseparowalnych - kara C (analogia do L2)
    
4. Funkcje straty